In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv
/kaggle/input/loan-approval-prediction/__results__.html
/kaggle/input/loan-approval-prediction/__output__.json
/kaggle/input/loan-approval-prediction/custom.css
/kaggle/input/loan-approval-prediction/__results___files/__results___20_1.png
/kaggle/input/loan-approval-prediction/__results___files/__results___18_0.svg
/kaggle/input/loan-approval-prediction/__results___files/__results___30_1.png
/kaggle/input/loan-approval-prediction/__results___files/__results___8_1.png
/kaggle/input/loan-approval-prediction/__results___files/__results___16_1.png
/kaggle/input/loan-approval-prediction/__results___files/__results___6_0.png
/kaggle/input/d/chilledwanker/loan-approval-prediction/credit_risk_dataset.csv


In [2]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [3]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import gc

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

import re

from functools import partial
from scipy.stats import kurtosis, skew, gmean, mode

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer, PowerTransformer, PolynomialFeatures,TargetEncoder,OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score, cross_val_predict, RepeatedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score, r2_score, accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.calibration import CalibrationDisplay, CalibratedClassifierCV
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Ridge, RidgeCV
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.svm import SVC, LinearSVR
from cuml.ensemble import RandomForestClassifier as rf
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
TargetEncoder
from sklearn.neural_network import MLPClassifier


import optuna

In [4]:
import pandas as pd
original=pd.read_csv('/kaggle/input/d/chilledwanker/loan-approval-prediction/credit_risk_dataset.csv')
original

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,N,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,N,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,N,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,N,26


In [5]:
X = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv', index_col='id')
y = X.pop('loan_status').astype('int8')
cat_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
X=pd.concat([X,original.drop(columns='loan_status')])
X.reset_index(inplace=True)
y=pd.concat([y,original['loan_status']])
y=y.reset_index()
y=y.loan_status
X.person_emp_length.fillna(X.person_emp_length.mean(),inplace=True)
X.loan_int_rate.fillna(X.loan_int_rate.mean(),inplace=True)
X.isnull().sum()
X=X.drop(columns='index')

In [6]:

def converting_datatypes(df, cat_cols, df_train=False):
    
    for col in cat_cols:
        df[col] = df[col].astype('category')
    
    df['person_age'] = df['person_age'].astype('int32')
    df['cb_person_cred_hist_length'] = df['cb_person_cred_hist_length'].astype('int32') 
    
    if df_train==True:

        df['loan_status'] = df['loan_status'].astype('int8')
        
    return df
def clip_data(df):
    
    df['person_age'] = df['person_age'].clip(None, 90)
    df['person_emp_length'] = df['person_emp_length'].clip(None, 60) 
    
    return df
cat_cols = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
X=converting_datatypes(X,X.select_dtypes(object).columns)
X=clip_data(X)
XX=X

In [7]:
log_transformer = FunctionTransformer(np.log1p, validate=True) 
preprocessing=ColumnTransformer([('cat',OneHotEncoder(drop='first', sparse_output=False),cat_cols),("num",log_transformer,['person_age','person_income','loan_amnt','person_emp_length','loan_percent_income'])],remainder='passthrough')

X=preprocessing.fit_transform(X)
df=pd.DataFrame(X)
'''  for i in df:
        plt.figure(figsize=(8, 6))  # Create a new figure for each plot
        sns.kdeplot(df[i], shade=True)
        plt.title(f'KDE Plot for {i}')
        plt.xlabel(i)
        plt.ylabel('Density')
        plt.show()'''


"  for i in df:\n        plt.figure(figsize=(8, 6))  # Create a new figure for each plot\n        sns.kdeplot(df[i], shade=True)\n        plt.title(f'KDE Plot for {i}')\n        plt.xlabel(i)\n        plt.ylabel('Density')\n        plt.show()"

In [8]:
'''import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score
import numpy as np

def objective(trial):
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1,log=True)  # Log scale
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees
    max_depth = trial.suggest_int('max_depth', -1, 2)  # Maximum depth of trees
    reg_alpha = trial.suggest_float('reg_alpha', 1e-3, 10.0,log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-3, 10.0,log=True)  # L2 regularization
    num_leaves = trial.suggest_int('num_leaves', 20, 50)  # Number of leaves
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.4, 1.0)  # Column sampling
    # Define the LightGBM model with suggested hyperparameters
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        num_leaves=num_leaves,
        colsample_bytree=colsample_bytree,
        verbose=-1,
        device='gpu'# Use all available cores
    )
    # Cross-validation strategy
    kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    
    # Scoring using AUC-ROC
    scoring = make_scorer(roc_auc_score)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=kfold, n_jobs=-1)
    acc = np.mean(scores)
    print(f"Trial {trial.number}: AUC-ROC = {acc:.5f}")
    return acc

# Create Optuna study and run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best trial
print("Best trial:")
trial = study.best_trial

print(f"AUC-ROC: {trial.value}")
print("Best hyperparameters: ", trial.params)'''


'import optuna\nfrom lightgbm import LGBMClassifier\nfrom sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score\nfrom sklearn.metrics import make_scorer, roc_auc_score\nimport numpy as np\n\ndef objective(trial):\n    learning_rate = trial.suggest_float(\'learning_rate\', 0.01, 0.1,log=True)  # Log scale\n    n_estimators = trial.suggest_int(\'n_estimators\', 50, 300)  # Number of trees\n    max_depth = trial.suggest_int(\'max_depth\', -1, 2)  # Maximum depth of trees\n    reg_alpha = trial.suggest_float(\'reg_alpha\', 1e-3, 10.0,log=True)  # L1 regularization\n    reg_lambda = trial.suggest_float(\'reg_lambda\', 1e-3, 10.0,log=True)  # L2 regularization\n    num_leaves = trial.suggest_int(\'num_leaves\', 20, 50)  # Number of leaves\n    colsample_bytree = trial.suggest_float(\'colsample_bytree\', 0.4, 1.0)  # Column sampling\n    # Define the LightGBM model with suggested hyperparameters\n    model = LGBMClassifier(\n        learning_rate=learning_rate,\n        n_es

In [9]:

'''def objective(trial):
    # Define hyperparameter search space
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1),
        'n_jobs': -1,
        'enable_categorical': True,'device':'gpu'
    }
    model=XGBClassifier(**xgb_params)
    kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    
    scoring = make_scorer(roc_auc_score)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=kfold, n_jobs=-1)
    acc = np.mean(scores)
    print(f"Trial {trial.number}: AUC-ROC = {acc:.5f}")
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial

print(f"AUC-ROC: {trial.value}")
print("Best hyperparameters: ", trial.params)'''


'def objective(trial):\n    # Define hyperparameter search space\n    xgb_params = {\n        \'n_estimators\': trial.suggest_int(\'n_estimators\', 50, 500),\n        \'max_depth\': trial.suggest_int(\'max_depth\', 3, 15),\n        \'learning_rate\': trial.suggest_float(\'learning_rate\', 0.01, 0.1),\n        \'gamma\': trial.suggest_float(\'gamma\', 0, 1),\n        \'min_child_weight\': trial.suggest_int(\'min_child_weight\', 1, 20),\n        \'colsample_bytree\': trial.suggest_float(\'colsample_bytree\', 0.3, 1),\n        \'n_jobs\': -1,\n        \'enable_categorical\': True,\'device\':\'gpu\'\n    }\n    model=XGBClassifier(**xgb_params)\n    kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)\n    \n    scoring = make_scorer(roc_auc_score)\n    scores = cross_val_score(model, X, y, scoring=\'roc_auc\', cv=kfold, n_jobs=-1)\n    acc = np.mean(scores)\n    print(f"Trial {trial.number}: AUC-ROC = {acc:.5f}")\n    return acc\n\nstudy = optuna.create_study(directio

In [10]:

'''def objective(trial):
    # Define hyperparameter search space
    max_features = trial.suggest_int('max_features', 1, X_train.shape[1])
    n_estimators = trial.suggest_int('n_estimators', 100, 2000)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    split_criterion = trial.suggest_categorical('split_criterion', ['gini', 'entropy'])
    n_bins = trial.suggest_int('n_bins', 16, 1024)

    # Create a RandomForest model using cuML
    model = rf(
        max_features=max_features,
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf,
        split_criterion=split_criterion,
        n_bins=n_bins,
        random_state=0
    )
    
    kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    scores = cross_val_score(model, X.astype('float32'), y, scoring='roc_auc', cv=kfold, n_jobs=-1)
    acc = np.mean(scores)
    print(f"Trial {trial.number}: AUC-ROC = {acc:.5f}")
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best trial:")
trial = study.best_trial

print(f"AUC-ROC: {trial.value}")
print("Best hyperparameters: ", trial.params)'''


'def objective(trial):\n    # Define hyperparameter search space\n    max_features = trial.suggest_int(\'max_features\', 1, X_train.shape[1])\n    n_estimators = trial.suggest_int(\'n_estimators\', 100, 2000)\n    min_samples_leaf = trial.suggest_int(\'min_samples_leaf\', 1, 10)\n    split_criterion = trial.suggest_categorical(\'split_criterion\', [\'gini\', \'entropy\'])\n    n_bins = trial.suggest_int(\'n_bins\', 16, 1024)\n\n    # Create a RandomForest model using cuML\n    model = rf(\n        max_features=max_features,\n        n_estimators=n_estimators,\n        min_samples_leaf=min_samples_leaf,\n        split_criterion=split_criterion,\n        n_bins=n_bins,\n        random_state=0\n    )\n    \n    kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)\n    scores = cross_val_score(model, X.astype(\'float32\'), y, scoring=\'roc_auc\', cv=kfold, n_jobs=-1)\n    acc = np.mean(scores)\n    print(f"Trial {trial.number}: AUC-ROC = {acc:.5f}")\n    return acc\n\n

In [11]:
kfold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)


params={'learning_rate': 0.08893469069331833, 'n_estimators': 300, 'max_depth': 0, 'reg_alpha': 1.5316600080462603, 'reg_lambda': 0.007814623028662943, 'num_leaves': 50, 'colsample_bytree': 0.9064898566204364}

base_model = LGBMClassifier(**params,verbose=-1)
base_model.fit(X,y)
scores = cross_val_score(base_model, X, y, scoring='roc_auc', cv=kfold, n_jobs=1)
print(f'{np.mean(scores):.5f} ± {np.std(scores):.5f}')


0.95976 ± 0.00231


In [12]:
ri_pred_lgb=base_model.predict_proba(X)[:,1]
ri_pred_lgb

array([0.03713879, 0.00244656, 0.00323516, ..., 0.95931827, 0.01776579,
       0.01193391])

In [13]:

rf_params={
    'max_features': 2,
    'n_estimators': 2000,
    'min_samples_leaf': 3,
    'split_criterion': 'entropy',
    'n_bins': 1024,
    'random_state': 0
}
base_model_rf = make_pipeline(
    TargetEncoder(random_state=0).set_output(transform='pandas'),
    FunctionTransformer(lambda XX: XX.astype(np.float32)),
    rf(**rf_params, n_streams=1, output_type='numpy')
)

scores = cross_val_score(base_model_rf, XX, y, scoring='roc_auc', cv=kfold, n_jobs=1)
print(f'{np.mean(scores):.5f} ± {np.std(scores):.5f}')

0.95998 ± 0.00341


In [14]:
base_model_rf.fit(XX,y)
ri_pred_rf=base_model_rf.predict_proba(converting_datatypes(XX,cat_cols))[:,1]
ri_pred_rf

array([0.01979791, 0.04059632, 0.0201671 , ..., 0.9320928 , 0.01307686,
       0.04491913], dtype=float32)

In [15]:
xgb_params={'n_estimators': 430, 'max_depth': 9, 'learning_rate': 0.04491829399123047, 'gamma': 0.24209212589557283, 'min_child_weight': 9, 'colsample_bytree': 0.5597656958749123}
base_model_xgb = XGBClassifier(**xgb_params)
base_model_xgb.fit(X,y)
scores = cross_val_score(base_model_xgb, X, y, scoring='roc_auc', cv=kfold, n_jobs=1)
print(f'{np.mean(scores):.5f} ± {np.std(scores):.5f}')


0.95725 ± 0.00231


In [16]:
ri_pred_xgb=base_model_rf.predict_proba(converting_datatypes(XX,cat_cols))[:,1]
ri_pred_rf

array([0.01979791, 0.04059632, 0.0201671 , ..., 0.9320928 , 0.01307686,
       0.04491913], dtype=float32)

In [17]:
test_df=pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv', index_col='id')
test_df1=converting_datatypes(test_df,cat_cols)
test_df=converting_datatypes(test_df,cat_cols)

In [18]:
cat_params = {
    'task_type'           : "CPU",
    'loss_function'       : 'Logloss',
    'eval_metric'         : "AUC",
    'bagging_temperature' : 0.25,
    'colsample_bylevel'   : 0.40,
    'iterations'          : 5_000,
    'learning_rate'       : 0.045,
    'max_depth'           : 7,
    'l2_leaf_reg'         : 0.80,
    'min_data_in_leaf'    : 30,
    'random_strength'     : 0.25,
    'random_state'        : 42,
    'early_stopping_rounds': 200,
    'allow_writing_files' : False,
} 

cat_cols_cat = ['person_age',
 'person_income',
 'person_home_ownership',
 'person_emp_length',
 'loan_intent',
 'loan_grade',
 'loan_amnt',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_default_on_file',
 'cb_person_cred_hist_length']

XX = XX.astype('str') 

test_pool = Pool(data=test_df.astype('str'), cat_features=cat_cols_cat)
model_pool = Pool(data=XX, label=y, cat_features=cat_cols_cat)
cat_model = CatBoostClassifier(**cat_params)
base_model_cat = make_pipeline(
      # Example of casting data if needed
    cat_model
)
#scores = cross_val_score(base_model_cat, model_pool, y, scoring='roc_auc', cv=kfold, n_jobs=1)

In [19]:
base_model_cat.fit(model_pool)

0:	total: 173ms	remaining: 14m 25s
1:	total: 268ms	remaining: 11m 9s
2:	total: 354ms	remaining: 9m 49s
3:	total: 441ms	remaining: 9m 10s
4:	total: 518ms	remaining: 8m 37s
5:	total: 610ms	remaining: 8m 27s
6:	total: 693ms	remaining: 8m 14s
7:	total: 789ms	remaining: 8m 12s
8:	total: 905ms	remaining: 8m 21s
9:	total: 987ms	remaining: 8m 12s
10:	total: 1.08s	remaining: 8m 10s
11:	total: 1.15s	remaining: 7m 59s
12:	total: 1.23s	remaining: 7m 53s
13:	total: 1.32s	remaining: 7m 52s
14:	total: 1.42s	remaining: 7m 52s
15:	total: 1.5s	remaining: 7m 46s
16:	total: 1.59s	remaining: 7m 47s
17:	total: 1.66s	remaining: 7m 40s
18:	total: 1.74s	remaining: 7m 36s
19:	total: 1.83s	remaining: 7m 35s
20:	total: 1.91s	remaining: 7m 33s
21:	total: 2.01s	remaining: 7m 34s
22:	total: 2.08s	remaining: 7m 30s
23:	total: 2.18s	remaining: 7m 32s
24:	total: 2.26s	remaining: 7m 30s
25:	total: 2.34s	remaining: 7m 27s
26:	total: 2.44s	remaining: 7m 29s
27:	total: 2.54s	remaining: 7m 30s
28:	total: 2.63s	remaining: 7m

Pipeline(steps=[('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x79a55960d3c0>)])

In [20]:
test_pool = Pool(data=test_df.astype('str'), cat_features=cat_cols_cat)
y_pred_cat=base_model_cat.predict_proba(test_pool)[:,1]
y_pred_cat.shape

(39098,)

In [21]:
test_df1=preprocessing.fit_transform(test_df)
y_pred_lgb=base_model.predict_proba(test_df1)
y_pred_xgb=base_model_xgb.predict_proba(test_df1)
y_pred_rf=base_model_rf.predict_proba(test_df)

In [22]:
#!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd

In [23]:
dfx=pd.DataFrame(X)
oof_preds_xgb = np.zeros(len(dfx))
oof_preds_lgb = np.zeros(len(dfx))
oof_preds_rf = np.zeros(len(dfx))
oof_preds_cat=np.zeros(len(dfx))

for i, (train_index, test_index) in enumerate(kfold.split(dfx, y)):
    X_train, X_test = dfx.iloc[train_index], dfx.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    XX_train,XX_test=XX.iloc[train_index], XX.iloc[test_index]
    
    test_pool = Pool(data=XX_test.astype('str'), cat_features=cat_cols_cat)
    model_pool = Pool(data=XX_train, label=y_train, cat_features=cat_cols_cat)
    lgb_mod = base_model.fit(X_train, y_train)
    xgb_mod = base_model_xgb.fit(X_train, y_train)
    rf_mod = base_model_rf.fit(XX_train, y_train)
    cat_mod=base_model_cat.fit(model_pool)
    
    preds_xgb = xgb_mod.predict_proba(X_test)[:, 1]  
    preds_lgb = lgb_mod.predict_proba(X_test)[:, 1]  
    preds_rf = rf_mod.predict_proba(XX_test)[:, 1]
    preds_cat=cat_mod.predict_proba(test_pool)[:,1]
    
    oof_preds_xgb[test_index] = preds_xgb
    oof_preds_lgb[test_index] = preds_lgb
    oof_preds_rf[test_index] = preds_rf
    oof_preds_cat[test_index]=preds_cat


    

0:	total: 107ms	remaining: 8m 56s
1:	total: 187ms	remaining: 7m 48s
2:	total: 286ms	remaining: 7m 56s
3:	total: 372ms	remaining: 7m 44s
4:	total: 466ms	remaining: 7m 45s
5:	total: 551ms	remaining: 7m 38s
6:	total: 635ms	remaining: 7m 32s
7:	total: 724ms	remaining: 7m 31s
8:	total: 800ms	remaining: 7m 23s
9:	total: 863ms	remaining: 7m 10s
10:	total: 946ms	remaining: 7m 9s
11:	total: 1.03s	remaining: 7m 8s
12:	total: 1.12s	remaining: 7m 10s
13:	total: 1.2s	remaining: 7m 6s
14:	total: 1.29s	remaining: 7m 7s
15:	total: 1.36s	remaining: 7m 5s
16:	total: 1.44s	remaining: 7m 2s
17:	total: 1.52s	remaining: 7m 2s
18:	total: 1.6s	remaining: 6m 59s
19:	total: 1.69s	remaining: 6m 59s
20:	total: 1.7s	remaining: 6m 43s
21:	total: 1.76s	remaining: 6m 38s
22:	total: 1.83s	remaining: 6m 35s
23:	total: 1.9s	remaining: 6m 33s
24:	total: 1.98s	remaining: 6m 33s
25:	total: 2.05s	remaining: 6m 33s
26:	total: 2.13s	remaining: 6m 31s
27:	total: 2.22s	remaining: 6m 33s
28:	total: 2.29s	remaining: 6m 32s
29:	to

In [24]:
def objective(trial):
    
    weight_xgb = trial.suggest_float('weight_xgb', 0, 1)
    weight_lgb = trial.suggest_float('weight_lgb', 0, 1)
    weight_rf = trial.suggest_float('weight_rf', 0, 1)
    weight_cat = trial.suggest_float('weight_cat', 0, 1)
   
    weight_sum = weight_xgb + weight_lgb+weight_rf+weight_cat
    weight_xgb /= weight_sum
    weight_lgb /= weight_sum
    weight_rf /= weight_sum
    weight_cat/=weight_sum
    
    blended_preds = weight_xgb * oof_preds_xgb**2 + weight_lgb * oof_preds_lgb**2+ weight_rf * oof_preds_rf**2+weight_cat * oof_preds_cat**2
    
    
    score = roc_auc_score(y, blended_preds)
    
    
    return -score



study = optuna.create_study(direction='minimize')


study.optimize(objective, n_trials=3000)

 
best_weights = study.best_params
print(f'Optimal Weights: XGBoost: {best_weights["weight_xgb"]}, LightGBM: {best_weights["weight_lgb"]},rf: {best_weights["weight_rf"]}')

[I 2024-10-24 15:11:54,473] A new study created in memory with name: no-name-4e1fbd0e-1e5d-470a-8968-4f6a8ae15c3f
[I 2024-10-24 15:11:54,514] Trial 0 finished with value: -0.9674306127806999 and parameters: {'weight_xgb': 0.5046317307261691, 'weight_lgb': 0.5264596647645708, 'weight_rf': 0.08028470798808651, 'weight_cat': 0.9907739326973256}. Best is trial 0 with value: -0.9674306127806999.
[I 2024-10-24 15:11:54,553] Trial 1 finished with value: -0.9667953359820819 and parameters: {'weight_xgb': 0.7602459717996304, 'weight_lgb': 0.6242588473857045, 'weight_rf': 0.5988483344788177, 'weight_cat': 0.5330652644686007}. Best is trial 0 with value: -0.9674306127806999.
[I 2024-10-24 15:11:54,593] Trial 2 finished with value: -0.9657484121128134 and parameters: {'weight_xgb': 0.6519137535283669, 'weight_lgb': 0.023320529918440247, 'weight_rf': 0.7602369793638855, 'weight_cat': 0.1491745045472298}. Best is trial 0 with value: -0.9674306127806999.
[I 2024-10-24 15:11:54,634] Trial 3 finished w

Optimal Weights: XGBoost: 0.01606939648575407, LightGBM: 0.13750231398218965,rf: 0.035734251403133854


In [25]:
'''def power_mean(preds, weights, power):
    weighted_preds = np.sum([w * (p ** power) for w, p in zip(weights, preds)], axis=0)
    return weighted_preds ** (1 / power)

def objective(trial):
    
    # Base model weights as parameters
    weight_xgb = trial.suggest_float('weight_xgb', 0, 1)
    weight_lgb = trial.suggest_float('weight_lgb', 0, 1)
    weight_rf = trial.suggest_float('weight_rf', 0, 1)
    weight_cat = trial.suggest_float('weight_cat', 0, 1)
    power = trial.suggest_float('power', -2, 2)  # Power parameter for power mean

    # Normalize weights
    weight_sum = weight_xgb + weight_lgb + weight_rf + weight_cat
    weight_xgb /= weight_sum
    weight_lgb /= weight_sum
    weight_rf /= weight_sum
    weight_cat /= weight_sum

    # Power mean blending
    blended_preds = power_mean([oof_preds_xgb, oof_preds_lgb, oof_preds_rf, oof_preds_cat], 
                               [weight_xgb, weight_lgb, weight_rf, weight_cat], 
                               power)

    score = roc_auc_score(y, blended_preds)
    
    return -score

study = optuna.create_study(direction='minimize')


study.optimize(objective, n_trials=3000)

 
best_weights = study.best_params
#print(f'Optimal Weights: XGBoost: {best_weights["weight_xgb"]}, LightGBM: {best_weights["weight_lgb"]},rf: {best_weights["weight_rf"]}')'''

'def power_mean(preds, weights, power):\n    weighted_preds = np.sum([w * (p ** power) for w, p in zip(weights, preds)], axis=0)\n    return weighted_preds ** (1 / power)\n\ndef objective(trial):\n    \n    # Base model weights as parameters\n    weight_xgb = trial.suggest_float(\'weight_xgb\', 0, 1)\n    weight_lgb = trial.suggest_float(\'weight_lgb\', 0, 1)\n    weight_rf = trial.suggest_float(\'weight_rf\', 0, 1)\n    weight_cat = trial.suggest_float(\'weight_cat\', 0, 1)\n    power = trial.suggest_float(\'power\', -2, 2)  # Power parameter for power mean\n\n    # Normalize weights\n    weight_sum = weight_xgb + weight_lgb + weight_rf + weight_cat\n    weight_xgb /= weight_sum\n    weight_lgb /= weight_sum\n    weight_rf /= weight_sum\n    weight_cat /= weight_sum\n\n    # Power mean blending\n    blended_preds = power_mean([oof_preds_xgb, oof_preds_lgb, oof_preds_rf, oof_preds_cat], \n                               [weight_xgb, weight_lgb, weight_rf, weight_cat], \n                

In [26]:
#y_preds=power_mean([y_pred_xgb[:,1], y_pred_lgb[:,1], y_pred_rf[:,1], y_pred_cat], [study.best_params['weight_xgb'], study.best_params['weight_lgb'],study.best_params['weight_rf'], study.best_params['weight_cat']], study.best_params['power'])


In [27]:
y_preds=study.best_params['weight_xgb']*y_pred_xgb[:,1]**2+study.best_params['weight_lgb']*y_pred_lgb[:,1]**2+study.best_params['weight_rf']*y_pred_rf[:,1]**2 +study.best_params['weight_cat']*y_pred_cat**2


In [28]:
results=pd.DataFrame({'id':test_df.index,'loan_status':y_preds})
results.to_csv('submission.csv',index=False)
